# CW-Husky SAD Triggering Demo

# Setup:

In [ ]:
PLATFORM = 'CW308_SAM4S'
SS_VER = "SS_VER_1_1"

In [ ]:
import chipwhisperer as cw

In [ ]:
%run ../../Setup_Scripts/Setup_Generic.ipynb

In [ ]:
scope.fpga_buildtime

# **Target #1: Software AES**

In [ ]:
cw.program_target(scope, prog, "firmware/simpleserial-aes-{}.hex".format(PLATFORM))

In [ ]:
reset_target(scope)

In [ ]:
scope.trigger.module = 'basic'
scope.trigger.triggers = 'tio4'

scope.adc.samples = 35000
scope.adc.presamples = 0
scope.adc.segments = 1
scope.adc.bits_per_sample = 8  # SAD is done at 8 bits per sample

In [ ]:
reftrace = cw.capture_trace(scope, target, bytearray(16), bytearray(16), as_int=True)

In [ ]:
refstart = 14570

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Span
output_notebook()

#p = figure(plot_width=1800)
p = figure(plot_width=1800, tools='pan, box_zoom, hover, reset, save')

xrange = range(len(reftrace.wave))
p.line(xrange, reftrace.wave)
p.renderers.extend([Span(location=refstart, dimension='height', line_color='black', line_width=2)])
p.renderers.extend([Span(location=refstart+scope.SAD.sad_reference_length, dimension='height', line_color='black', line_width=2)])

show(p)

# Calculating SAD in software
- very slow! Husky hardware does this in the blink of an eye
- SW model is configured with `scope.SAD`, so there is no need to transfer model specs to the hardware scope 

In [ ]:
scope.SAD.reference = reftrace.wave[refstart:]
scope.SAD.threshold = 30
scope.SAD.interval_threshold = 20
scope.SAD.multiple_triggers = True
scope.SAD.emode = False

In [ ]:
sad_model = cw.SADModelWrapper(scope.SAD, catch_emisses=False, verbose=False)
sad_model.run(reftrace.wave)
print(sad_model)

We should have matched on each of the 10 rounds:

In [ ]:
assert len(sad_model.match_times) == 10

In [ ]:
p = figure(plot_width=1800)
dat = sad_model.SADS

xrange = range(len(dat))
p.line(xrange, dat)
p.renderers.extend([Span(location=scope.SAD.threshold, dimension='width', line_color='red', line_width=2)])
show(p)

# Hardware SAD Triggering with the SAD Explorer

In [ ]:
scope.trigger.module = 'SAD'
scope.adc.samples = 300
scope.adc.segments = 10
scope.adc.timeout = 0.1

In [ ]:
explorer = cw.SADExplorer(scope, target, reftrace.wave, refstart, max_segments=10)

### Things to try:
1. The first round has a few samples at the start that diverge considerably from the reference; exclude those samples and make the thresholds tighter. The "show diff" option can be helpful here.
2. Turn on "emode".
3. What happens when you make the thresholds much too loose, or exclude too many samples?
4. Increase `scope.adc.samples` and `scope.adc.presamples` to hunt around for a better `refstart` nearby.
5. How small can you make `scope.SAD.trigger_sample` and still reliably capture all 10 rounds? *(be sure to check that the trigger times are still good)*
6. Try to find a totally different reference segment that works well.

*known good parameters in hidden cell below in case the notebook gets messed up:*

In [ ]:
refstart = 14570
scope.SAD.threshold = 30 # increase for emode due to half-threshold effect
scope.SAD.interval_threshold = 20
scope.SAD.emode = False
scope.SAD.always_armed = False
scope.SAD.reference = reftrace.wave[refstart:]

scope.adc.stream_mode = False
scope.adc.samples = 300
scope.adc.segments = 10

In [ ]:
# For #5 (advanced trigger), these are nice settings:
refstart = 14610
scope.SAD.emode = False
scope.SAD.threshold = 3
scope.SAD.interval_threshold = 20
scope.SAD.trigger_sample = 56

In [ ]:
# Using random key/plaintext:
ktp = cw.ktp.Basic()
key, text = ktp.next()
explorer = cw.SADExplorer(scope, target, reftrace.wave, refstart, max_segments=10, capture_function=lambda: cw.capture_trace(scope, target, key, text, as_int=True))

# **Target #2: Software ECC**

In [ ]:
cw.program_target(scope, prog, "firmware/simpleserial-ecc-fwtrigger-{}.hex".format(PLATFORM))
reset_target(scope)

In [ ]:
target.simpleserial_write('i', b'')
time.sleep(0.1)
print(target.read())

In [ ]:
TRACES = 'HARDWARE'
%run "../../courses/sca205/ECC_capture.ipynb"

In [ ]:
scope.trigger.module = 'basic'
scope.trigger.triggers = 'tio4'

scope.adc.stream_mode = True
scope.adc.presamples = 0
scope.adc.samples = int(16e6)
scope.adc.segments = 1

The micro-ecc target firmware is not constant time; in order to have a known good SAD reference, we stick to these parameters:

In [ ]:
k = 0x526a13ac66957d13622a9d872ff9302c47d6393237efaa4c0fc92c08febc5d2c
Px = 0xe479bb253840235126427b2cdff9a862601e1577c2abbc274d4b5372a45656ec
Py = 0x561fbeb30f276006b91ba1b81df8e3f3edf40f8ea000593b3a622610af02a50

In [ ]:
# try different k / Px / Py!
#k = random_k()
#Px, Py = new_point()

In [ ]:
reftrace = capture_ecc_trace(k, Px, Py)
scope.errors.clear()
print(scope.adc.trig_count)
assert scope.adc.trig_count == 15788560

In [ ]:
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade
hv.extension('bokeh')
datashade(hv.Curve(reftrace.wave)).opts(width=2000, height=900)

The target firmware triggers at the start of each bit being processed by the point multiplication algorithm; let's peak at how long each bit takes:

In [ ]:
ttimes = scope.trigger.get_trigger_times()

In [ ]:
print(ttimes[:10])
print('Min: %d' % min(ttimes))
print('Max: %d' % max(ttimes))

# sanity check:
assert min(ttimes) == 105024 and max(ttimes) == 110900

In [ ]:
scope.SAD.emode = True
refstart = 1028600

Let's plot a subset of the reference trace that covers about 3 bits somewhere in the middle of the operation:

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Span

start = int(1e6)
samples = 300000
output_notebook()
p = figure(plot_width=1800, tools='pan, box_zoom, hover, reset, save')

xrange = range(samples)
p.line(xrange, reftrace.wave[start:start+samples])

p.renderers.extend([Span(location=refstart-start, dimension='height', line_color='black', line_width=1)])
p.renderers.extend([Span(location=refstart-start+scope.SAD.sad_reference_length, dimension='height', line_color='black', line_width=1)])

show(p)

In [ ]:
scope.trigger.module = 'SAD'
scope.SAD.threshold = 15
scope.SAD.interval_threshold = 20
scope.SAD.emode = True
scope.SAD.always_armed = False
scope.SAD.multiple_triggers = True
scope.SAD.reference = reftrace.wave[refstart:]
scope.adc.stream_mode = False

In [ ]:
explorer = cw.SADExplorer(scope, target, reftrace.wave, refstart, max_segments=255, capture_function=lambda: capture_ecc_trace(k, Px, Py))

## Things to try:
1. Turn on `scope.adc.always_armed` to make sure that 255 matches are seen.
2. If you turn off `scope.SAD.emode`, you can capture all 255 segments.
3. How few samples are needed to reliably match? (reduce `scope.SAD.trigger_sample`).
4. We used a specific k/Px/Py to find our SAD reference. If you change any/all of k/Px/Py, does that reference still work?
5. Can you find another suitable SAD reference?

In [ ]:
# to use different k / Px / Py:
k = random_k()
Px, Py = new_point()

*known good parameters in hidden cell below:*

In [ ]:
refstart = 1028600

# re-acquire reftrace if these changed!
k = 0x526a13ac66957d13622a9d872ff9302c47d6393237efaa4c0fc92c08febc5d2c
Px = 0xe479bb253840235126427b2cdff9a862601e1577c2abbc274d4b5372a45656ec
Py = 0x561fbeb30f276006b91ba1b81df8e3f3edf40f8ea000593b3a622610af02a50

scope.SAD.threshold = 15
scope.SAD.interval_threshold = 20
scope.SAD.emode = True
scope.SAD.always_armed = False
scope.SAD.reference = reftrace.wave[refstart:]

scope.adc.stream_mode = False
scope.adc.samples = 450
scope.adc.segments = 200